# SWF012-T1 test: Image cube inspection

This notebook is an example of the test case for SWF-012. The test is described in more details at https://confluence.skatelescope.org/pages/viewpage.action?pageId=319991468 and aims at evaluating the SRCNet v0.1 pipeline's ability to inspect image cubes and do some simple operations on them such as computing statistics, generating histograms, zoom on some regions, perform 2D Gaussian fitting. The test case is divided in sub-cases, corresponding to each aforementioned task.


Note that the datasets used in this notebook must be downloaded at the following link: https://www.dropbox.com/sh/22cbwyy8hv7h4fq/AACx6cTQ1SN8ldmcu6P4Fj9ua?dl=0 (10GB).

In [1]:
# import required packages
import numpy as np
import warnings
import time
import os
import matplotlib.pyplot as plt
from matplotlib import colors
import scipy.ndimage
from astropy.io import fits
from astropy import units, constants
from astropy.wcs import WCS
from astropy.visualization import simple_norm
from astropy.convolution import convolve, convolve_fft, Gaussian2DKernel
from astropy.modeling.models import Gaussian2D
from astropy.wcs.utils import proj_plane_pixel_scales
from astropy.nddata.utils import Cutout2D
from astropy.wcs.utils import pixel_to_skycoord

In [3]:
# Optional: use inline if not working
# %matplotlib widget
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [4]:
datafolder = 'swf012_data/'  # where to find the downloaded data

## Load data cubes from FITS files

We choose a limited spectral window to reduce computations:

In [11]:
df = 0.100  # MHz, frequency resolution
fmin = 166.0  # minimum frequency on the spectral window
fmax = 180.9  # maximum frequency on the spectral window
spw_range = np.round(np.arange(fmin, fmax+df, step=df), decimals=2) * units.MHz  # spectral window
avg_nu = np.mean(spw_range)  # mean frequency over spectral window
avg_z = nu0/avg_nu - 1.  # mean redshift over spectral window
lamb_array = constants.c.si / (spw_range.si)  # array of observed wavelengths over spectral window

Below, we extract the required frequency channels from the input files and save the result as a new fits file which will be fed to `ps_eor`.

In [12]:
def reduce_file(filename, spw_range):
    """
    Method to select given frequency channels in FITS file.

    Parameters
    ----------
        filename: str
            Path to the FITS files containing the image data (3D).
            Should have units (astropy.units).
        spw_range: array of floats
            List of frequencies to extract from dataset.
    
    Returns
    ------
        hdu
    """
    nf = spw_range.size
    hdu = fits.open(filename)
    nfreqs = hdu[0].header['NAXIS3']
    df = hdu[0].header['CDELT3'] * units.Hz
    fmin = hdu[0].header['CRVAL3'] * units.Hz
    fmax = fmin + df * nfreqs
    freqs = np.arange(fmin.value, fmax.value, step=df.value) * units.Hz
    assert fmin <= spw_range[0].to(units.Hz)
    assert fmax >= spw_range[-1].to(units.Hz)
    assert freqs.size == nfreqs
    inds = np.where((freqs<=spw_range[-1]) & (freqs>=spw_range[0]))[0]
    assert inds.size == nf
    hdu[0].data = hdu[0].data[inds, :, :]
    hdu[0].header['NAXIS3'] = nf
    hdu[0].header['CRVAL3'] = spw_range[0].to(units.Hz).value
    return hdu

In [13]:
# REDUCING FILES
weighting = 'msn'

# PSF
reduced_psf_file = f'{datafolder}TestDataset.{weighting}_psf_reduced.fits'
if os.path.exists(reduced_psf_file):
    # If the reduced file exist, make sure it contains the appropriate frequencies.
    hdu = fits.open(reduced_psf_file)
    du = abs(hdu[0].header['CDELT1'])
    assert hdu[0].data.shape[0] == spw_range.size
    hdu.close()
else:
    # Else, load full PSF file and reduce
    psffile = f'{datafolder}TestDataset.{weighting}_psf.fits'
    hdu = reduce_file(psffile, spw_range)
    du = abs(hdu[0].header['CDELT1'])
    npix = hdu[0].data.shape[1]
    # Save to new file
    hdu.writeto(reduced_psf_file, overwrite=True)
    hdu.close()

# IMAGE
reduced_image_file = f'{datafolder}IM1.{weighting}_image_reduced.fits'
if os.path.exists(reduced_image_file):
    # If the reduced file exist, make sure it contains the appropriate frequencies.
    hdu = fits.open(reduced_image_file)
    assert hdu[0].data.shape[0] == spw_range.size
    hdu.close()
else:
    # Else, load full image file and reduce.
    imagefile = f'{datafolder}IM1.{weighting}_image.fits' # location of the image file downloaded from SKA SDC3 drive
    hdu = reduce_file(imagefile, spw_range)
    hdu.writeto(reduced_image_file, overwrite=True)
hdu.close()

## SWF12.1 Create regions

## SWF12.2 Compute image statistics

## SWF 12.3 Generate image histograms

## SWF 12.4 2D gaussian fitting

## SWF 12.5 Change of reference frame for the spectral/velocity axis